Data Needed from nba_api:

Per-Player:
- General data: full name, number, team, and position
- Season per-game stats: PTS, AST, REB, STL, BLK, TO, FG% and FGA/FGM, FT% and FTM/FTA, 3P% and 3PM/3PA, GP, MPG
- Season per game advanced stats: PER, +/-, TS%, WS

In [29]:
# Query nba.live.endpoints.scoreboard and  list games in localTimeZone
from datetime import datetime, timezone
from dateutil import parser
# !pip install nba_api
from nba_api.live.nba.endpoints import scoreboard

f = "{gameId}: {awayTeam} vs. {homeTeam} @ {gameTimeLTZ}" 

board = scoreboard.ScoreBoard()
print("ScoreBoardDate: " + board.score_board_date)
games = board.games.get_dict()
for game in games:
    gameTimeLTZ = parser.parse(game["gameTimeUTC"]).replace(tzinfo=timezone.utc).astimezone(tz=None)
    print(f.format(gameId=game['gameId'], awayTeam=game['awayTeam']['teamName'], homeTeam=game['homeTeam']['teamName'], gameTimeLTZ=gameTimeLTZ))

from nba_api.live.nba.endpoints import boxscore
box = boxscore.BoxScore('0022500127') 

players = box.away_team.get_dict()['players']
f = "{player_id}: {name}: {points} PTS"
for player in players:
    print(f.format(player_id=player['personId'],name=player['name'],points=player['statistics']['points']))
    print(player)

ConnectionError: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))

In [37]:
from __future__ import annotations

import os
import time
from datetime import date, datetime
from typing import Dict, List, Optional, Tuple

import pandas as pd

try:
    import requests_cache  # type: ignore
except Exception:
    requests_cache = None  # type: ignore

from nba_api.stats.library.http import NBAStatsHTTP
from nba_api.stats.endpoints import (
    leaguedashplayerstats,
    teamdashboardbygeneralsplits,
    scoreboardv2,
    boxscoretraditionalv2,
    leaguegamelog,
)
from nba_api.stats.static import teams as static_teams

def _retry_call(func, *args, retries: int = 3, delay: float = 1.0, **kwargs):
    last_exc = None
    for attempt in range(retries):
        try:
            return func(*args, **kwargs)
        except Exception as e:  # noqa: BLE001
            last_exc = e
            if attempt < retries - 1:
                time.sleep(delay * (attempt + 1))
    if last_exc:
        raise last_exc

In [31]:
def current_season_str(today: Optional[date] = None) -> str:
    """Return NBA season string like '2024-25'. Season rolls in October."""
    today = today or date.today()
    start_year = today.year if today.month >= 10 else today.year - 1
    return f"{start_year}-{str(start_year + 1)[-2:]}"

season = current_season_str()

print(f"Season: {season}")

Season: 2025-26


In [40]:
# 1) Current season stats for each player
def fetch_players_per_game(season: str, season_type: str = "Regular Season") -> pd.DataFrame:
    """Fetch per-game box stats for all players for a season."""
    resp = _retry_call(
        leaguedashplayerstats.LeagueDashPlayerStats,
        season=season,
        season_type_all_star=season_type,
        per_mode_detailed="PerGame",
        timeout=30,
    )
    df = resp.get_data_frames()[0]
    # Ensure predictable dtypes for common fields
    cols = ["PTS", "AST", "REB", "STL", "BLK", "FGA", "FGM", "FG_PCT", "FG3M", "FG3A", "FTM", "FTA", "MIN"]
    for col in cols:
        if col in df.columns:
            df[col] = pd.to_numeric(df[col], errors="coerce")
    # return df[[c for c in cols if c in df.columns]].copy()
    return df

players = fetch_players_per_game(season)
# pd.set_option('display.max_columns', None)
players



,PLAYER_ID,PLAYER_NAME,NICKNAME,TEAM_ID,TEAM_ABBREVIATION,AGE,GP,W,L,W_PCT,MIN,FGM,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,FTM,FTA,FT_PCT,OREB,DREB,REB,AST,TOV,STL,BLK,BLKA,PF,PFD,PTS,PLUS_MINUS,NBA_FANTASY_PTS,DD2,TD3,WNBA_FANTASY_PTS,GP_RANK,W_RANK,L_RANK,W_PCT_RANK,MIN_RANK,FGM_RANK,FGA_RANK,FG_PCT_RANK,FG3M_RANK,FG3A_RANK,FG3_PCT_RANK,FTM_RANK,FTA_RANK,FT_PCT_RANK,OREB_RANK,DREB_RANK,REB_RANK,AST_RANK,TOV_RANK,STL_RANK,BLK_RANK,BLKA_RANK,PF_RANK,PFD_RANK,PTS_RANK,PLUS_MINUS_RANK,NBA_FANTASY_PTS_RANK,DD2_RANK,TD3_RANK,WNBA_FANTASY_PTS_RANK,TEAM_COUNT
0,1631260,AJ Green,AJ,1610612749,MIL,26.0,4,3,1,0.750,26.8,4.0,6.3,0.640,3.3,5.5,0.591,1.0,1.3,0.800,0.0,2.5,2.5,0.8,1.3,0.3,0.0,0.0,3.3,1.5,12.3,8.3,15.9,0,0,19.3,9,28,209,84,127,137,192,52,14,68,28,180,217,154,315,182,234,277,166,279,225,237,63,179,120,38,217,65,3,174,1
1,1642358,AJ Johnson,AJ,1610612764,WAS,20.0,2,0,2,0.000,2.9,0.0,0.5,0.000,0.0,0.0,0.000,0.0,0.0,0.000,0.0,0.5,0.5,0.5,0.0,0.0,0.0,0.0,0.5,0.0,0.0,-1.0,1.4,0,0,1.0,297,336,92,336,390,375,379,375,287,340,287,306,311,306,315,334,356,297,343,304,225,237,342,337,383,207,387,65,3,389,1
2,203932,Aaron Gordon,Aaron,1610612743,DEN,30.0,3,2,1,0.667,32.2,8.3,14.0,0.595,3.7,6.3,0.579,5.0,5.0,1.000,0.3,4.0,4.3,2.0,1.3,1.0,0.3,0.3,1.3,4.3,25.3,8.0,36.2,0,0,38.0,191,76,209,118,61,24,51,71,8,46,30,32,49,1,276,92,135,145,153,95,146,168,262,43,22,40,44,65,3,35,1
3,1628988,Aaron Holiday,Aaron,1610612745,HOU,29.0,1,1,0,1.000,7.2,2.0,3.0,0.667,1.0,2.0,0.500,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,-1.0,5.0,0,0,6.0,370,195,336,1,334,235,292,39,159,224,46,306,311,306,315,361,374,344,343,304,225,237,368,337,258,207,340,65,3,321,1
4,1630174,Aaron Nesmith,Aaron,1610612754,IND,26.0,3,0,3,0.000,31.1,4.3,11.7,0.371,3.3,7.0,0.476,0.7,1.7,0.400,1.3,4.0,5.3,1.0,0.3,0.0,0.3,1.0,3.0,3.0,12.7,-2.3,21.2,0,0,23.0,191,336,9,336,83,123,86,293,13,28,74,242,174,292,99,92,96,230,312,304,146,23,71,80,115,257,161,65,3,133,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
413,1642258,Zaccharie Risacher,Zaccharie,1610612737,ATL,20.0,2,0,2,0.000,20.5,4.0,9.0,0.444,1.0,4.0,0.250,0.0,0.0,0.000,1.0,0.0,1.0,0.0,0.5,0.0,0.5,0.5,1.0,0.0,9.0,-0.5,11.2,0,0,12.0,297,336,92,336,204,137,133,219,159,124,236,306,311,306,129,361,318,344,280,304,98,115,292,337,173,198,270,65,3,253,1
414,203897,Zach LaVine,Zach,1610612758,SAC,30.0,4,1,3,0.250,36.7,10.3,19.8,0.519,3.5,8.5,0.412,5.0,5.3,0.952,0.3,2.8,3.0,1.3,2.3,0.0,0.3,0.8,2.0,3.8,29.0,-6.8,33.0,0,0,37.3,9,195,9,272,11,11,9,133,9,11,109,32,45,85,293,167,199,222,59,304,176,63,179,53,12,360,65,65,3,38,1
415,1630192,Zeke Nnaji,Zeke,1610612743,DEN,24.0,2,2,0,1.000,2.2,0.0,0.5,0.000,0.0,0.5,0.000,1.0,1.0,1.000,0.0,0.0,0.0,0.5,0.0,0.0,0.0,0.0,0.5,1.0,1.0,-1.0,1.8,0,0,1.5,297,76,336,1,396,375,379,375,287,315,287,180,226,1,315,361,374,297,343,304,225,237,342,221,364,207,381,65,3,384,1
416,1630533,Ziaire Williams,Ziaire,1610612751,BKN,24.0,3,0,3,0.000,18.5,3.3,6.7,0.500,2.3,5.0,0.467,1.3,1.7,0.800,1.0,1.0,2.0,0.3,0.0,1.3,0.0,0.3,3.3,1.3,10.3,-2.3,17.2,0,0,17.7,191,336,9,336,230,165,183,142,54,87,79,157,174,154,129,291,259,325,343,52,225,168,54,198,147,257,195,65,3,189,1


In [ ]:
# 2) Current season team stats (advanced)
def fetch_teams_advanced(season: str, season_type: str = "Regular Season") -> pd.DataFrame:
    """Fetch team advanced stats (OFF/DEF/NET RTG, Pace, record) via TeamDashboardByGeneralSplits.

    This endpoint returns a single row per team with OFF_RATING, DEF_RATING, NET_RATING, PACE, W, L, etc.
    """
    # Get teams (static)
    teams = static_teams.get_teams()
    rows: List[pd.DataFrame] = []
    for t in teams:
        team_id = t["id"]
        resp = _retry_call(
            teamdashboardbygeneralsplits.TeamDashboardByGeneralSplits,
            team_id=team_id,
            season=season,
            season_type_all_star=season_type,
            per_mode_detailed="PerGame",
            timeout=30,
        )
        df = resp.overall_team_dashboard.get_data_frame()
        # Attach extra identifiers
        df["TEAM_ID"] = team_id
        df["TEAM_ABBREVIATION"] = t["abbreviation"]
        df["TEAM_NAME"] = t["full_name"]
        rows.append(df)

    out = pd.concat(rows, ignore_index=True)
    # Select/rename to a friendly schema
    cols = [
        "TEAM_ID",
        "TEAM_ABBREVIATION",
        "TEAM_NAME",
        "W",
        "L",
        "W_PCT",
        "GP",
        "MIN",
        "PLUS_MINUS",
        # "OFF_RATING",
        # "DEF_RATING",
        # "NET_RATING",
        # "PACE",
    ]
    return out[[c for c in cols if c in out.columns]].copy()
    # return out.copy()

teams_adv = fetch_teams_advanced(season)
teams_adv

,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,W,L,W_PCT,GP,MIN,PLUS_MINUS
0,1610612737,ATL,Atlanta Hawks,4,1,0.800,5,40.4,2.2
1,1610612737,ATL,Atlanta Hawks,1,3,0.250,4,48.1,-9.5
2,1610612738,BOS,Boston Celtics,4,1,0.800,5,40.0,10.6
3,1610612738,BOS,Boston Celtics,1,3,0.250,4,264.1,-0.8
4,1610612739,CLE,Cleveland Cavaliers,2,3,0.400,5,40.0,2.0
5,1610612739,CLE,Cleveland Cavaliers,3,1,0.750,4,48.0,6.3
6,1610612740,NOP,New Orleans Pelicans,0,5,0.000,5,40.0,-9.8
7,1610612740,NOP,New Orleans Pelicans,0,3,0.000,3,49.7,-14.0
8,1610612741,CHI,Chicago Bulls,3,2,0.600,5,40.0,-6.6
9,1610612741,CHI,Chicago Bulls,3,0,1.000,3,48.0,7.0


: 

In [35]:
# 3) Box score stats for today's games
def fetch_scoreboard(game_date: Optional[date] = None) -> Tuple[pd.DataFrame, pd.DataFrame, List[str]]:
    """Return (game_header, line_score, game_ids) for a given date (defaults to today)."""
    d = game_date or date.today()
    datestr = d.strftime("%m/%d/%Y")
    resp = _retry_call(scoreboardv2.ScoreboardV2, game_date=datestr, timeout=30)
    game_header = resp.game_header.get_data_frame()
    line_score = resp.line_score.get_data_frame()
    game_ids = list(game_header["GAME_ID"].astype(str).unique())
    return game_header, line_score, game_ids


def fetch_boxscores_for_games(game_ids: List[str]) -> Dict[str, pd.DataFrame]:
    """Fetch player-level traditional box score for each game_id.

    Returns dict[game_id] -> DataFrame
    """
    out: Dict[str, pd.DataFrame] = {}
    for gid in game_ids:
        resp = _retry_call(boxscoretraditionalv2.BoxScoreTraditionalV2, game_id=gid, timeout=30)
        df = resp.player_stats.get_data_frame()
        out[gid] = df
    return out


# 4) General stats for every game in season
def fetch_season_games_summary(season: str, season_type: str = "Regular Season") -> pd.DataFrame:
    """Fetch season-to-date team game logs and roll up to one row per GAME_ID with
    home/away teams, scores, and date.

    Uses LeagueGameLog with player_or_team_abbreviation='T'. Each game appears twice (one per team).
    """
    resp = _retry_call(
        leaguegamelog.LeagueGameLog,
        season=season,
        season_type_all_star=season_type,
        player_or_team_abbreviation="T",
        timeout=30,
    )
    logs = resp.get_data_frames()[0]
    # Normalize types
    for col in ("PTS",):
        if col in logs.columns:
            logs[col] = pd.to_numeric(logs[col], errors="coerce")

    # Determine home/away from MATCHUP ("TEAM vs OPP" -> home, "TEAM @ OPP" -> away)
    logs["IS_HOME"] = logs["MATCHUP"].str.contains(" vs ")

    # Split into home and away rows then merge per GAME_ID
    home = logs[logs["IS_HOME"]].copy()
    away = logs[~logs["IS_HOME"]].copy()

    # Prepare columns
    home = home.rename(columns={
        "TEAM_NAME": "HOME_TEAM",
        "TEAM_ABBREVIATION": "HOME_ABBR",
        "PTS": "HOME_PTS",
        "GAME_DATE": "GAME_DATE",
        "GAME_ID": "GAME_ID",
    })
    away = away.rename(columns={
        "TEAM_NAME": "AWAY_TEAM",
        "TEAM_ABBREVIATION": "AWAY_ABBR",
        "PTS": "AWAY_PTS",
        "GAME_ID": "GAME_ID",
    })

    merged = pd.merge(
        home[["GAME_ID", "GAME_DATE", "HOME_TEAM", "HOME_ABBR", "HOME_PTS"]],
        away[["GAME_ID", "AWAY_TEAM", "AWAY_ABBR", "AWAY_PTS"]],
        on="GAME_ID",
        how="inner",
        validate="one_to_one",
    )

    # Coerce date
    try:
        merged["GAME_DATE"] = pd.to_datetime(merged["GAME_DATE"]).dt.date
    except Exception:
        pass

    return merged.sort_values(["GAME_DATE", "GAME_ID"]).reset_index(drop=True)



print("Fetching today's scoreboard + box scores…")
gh, ls, game_ids = fetch_scoreboard()
print(f"Game header: {gh}, Line score: {ls}, Game IDs: {game_ids}")
boxes = fetch_boxscores_for_games(game_ids)
for gid, df in boxes.items():
    print("Game ID: ", gid)
    print(df)

print("Fetching season game summaries…")
games = fetch_season_games_summary(season)
print(games)

Fetching today's scoreboard + box scores…


ReadTimeout: HTTPSConnectionPool(host='stats.nba.com', port=443): Read timed out. (read timeout=30)